In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from client import Client
from webhook_listener import WebhookListener

In [15]:
client = Client(url="http://127.0.0.1:8000")
client_l1 = Client(url="http://127.0.0.1:8000", l1_key="abc")
client_l2 = Client(url="http://127.0.0.1:8000", l2_key="def")

In [18]:
client.create_user("pc3", balance="1080.50")

{'name': 'pc3', 'balance': '1080.50'}

In [19]:
client.create_user("pc1", balance="10811.50")

{'name': 'pc1', 'balance': '10811.50'}

In [22]:
client.get_user("pc3")

{'name': 'pc3', 'balance': '1080.50'}

In [5]:
client_l1.reset_user_balance("pc3", "500.00")

{'name': 'pc3', 'balance': '500.00'}

In [18]:
client.buy(
    user_name="pc3",
    market="0xcccb7e7613a087c132b69cbf3a02bece3fdcb824c1da54ae79acc8d4a562d902",
    token="8441400852834915183759801017793514978104486628517653995211751018945988243154",
    amount_usdc="400.00"
)

{'status': 'success',
 'details': {'amount_usdc': '400.00',
  'shares': '530.71',
  'average_price': '0.7537072977709106668425317028',
  'fills': 4}}

In [19]:
client.sell(
    user_name="pc3",
    market="0xcccb7e7613a087c132b69cbf3a02bece3fdcb824c1da54ae79acc8d4a562d902",
    token="8441400852834915183759801017793514978104486628517653995211751018945988243154",
    shares="1.00"
)

{'status': 'success',
 'details': {'amount_usdc': '0.71',
  'shares': '1.00',
  'average_price': '0.71',
  'fills': 1}}

In [4]:
client.list_orders()

[{'user_name': 'pc3',
  'market': '0xcccb7e7613a087c132b69cbf3a02bece3fdcb824c1da54ae79acc8d4a562d902',
  'token': '8441400852834915183759801017793514978104486628517653995211751018945988243154',
  'side': 'BUY',
  'order_type': 'MARKET',
  'status': 'FILLED',
  'amount_usdc': '400.00',
  'shares': '530.71',
  'created_at': '2025-08-09T05:04:31.496427',
  'updated_at': '2025-08-09T05:04:31.496427'},
 {'user_name': 'pc3',
  'market': '0xcccb7e7613a087c132b69cbf3a02bece3fdcb824c1da54ae79acc8d4a562d902',
  'token': '8441400852834915183759801017793514978104486628517653995211751018945988243154',
  'side': 'SELL',
  'order_type': 'MARKET',
  'status': 'FILLED',
  'amount_usdc': '0.71',
  'shares': '1.00',
  'created_at': '2025-08-09T05:06:03.658971',
  'updated_at': '2025-08-09T05:06:03.658971'}]

In [7]:
client.list_orders_by_user("pc1")

[]

In [12]:
client.list_positions()

[{'market': '0xcccb7e7613a087c132b69cbf3a02bece3fdcb824c1da54ae79acc8d4a562d902',
  'token': '8441400852834915183759801017793514978104486628517653995211751018945988243154',
  'shares': '529.71'}]

In [13]:
client.list_positions_by_user("pc1")

[]

In [14]:
client.list_positions_by_user("pc3")

[{'market': '0xcccb7e7613a087c132b69cbf3a02bece3fdcb824c1da54ae79acc8d4a562d902',
  'token': '8441400852834915183759801017793514978104486628517653995211751018945988243154',
  'shares': '529.71'}]

In [16]:
client_l2.delete_all_data()

{'success': True, 'message': 'All data cleared.'}

In [20]:
client_l2.exec_sql(
    "SELECT * FROM users",
    limit=100,
)


{'columns': ['name', 'balance'],
 'rows': [{'name': 'pc3', 'balance': 1080.5},
  {'name': 'pc1', 'balance': 10811.5}],
 'truncated': False}

In [4]:
wl = WebhookListener(port=8001, path="/market-event")

In [5]:
def on_market_added(data: dict):
    # data: {"markets": [...]}
    print("[market_added]", len(data.get("markets", [])))

def on_market_resolved(data: dict):
    # data: {"markets": [...]}
    print("[market_resolved]", data)

# If they don't care about payout logs, they simply don't subscribe.
def on_payout_logs(data: dict):
    print("[payout_logs]", len(data.get("payout_logs", [])))

wl.on("market_added", on_market_added)
wl.on("market_resolved", on_market_resolved)
wl.on("payout_logs", on_payout_logs)

In [6]:
wl.start()
input("Listening… press Enter to stop.\n")
wl.stop()

Listening on http://0.0.0.0:8001/market-event
